In [ ]:
###Changing the RSME code that Sammi started last semester for future work in the new semester

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)
#######################
#from src import wdata
from netCDF4 import Dataset
import seaborn as sns;
%matplotlib nbagg
from skimage import io, measure, transform

In [4]:
def rgb2gray(rgb):
    """ Only temporary. Need to use inverse of function used to save radar to RGB .pngs"""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [5]:
def readFolder(basedir):
    
    from bs4 import BeautifulSoup as bs
    bshtml = None
    with open(basedir+'index.html', 'rb') as f:
        bshtml = bs(f, 'html.parser')
    tds = bshtml.find_all('td')
    inp, out, targ = [], [], []
    for i in range(0,len(tds),4):
        inp.append(tds[i+1].contents[0]['src'])
        out.append(tds[i+2].contents[0]['src'])
        targ.append(tds[i+3].contents[0]['src'])

    inp = map(lambda x: rgb2gray(plt.imread(basedir+x)), inp)
    out = map(lambda x: rgb2gray(plt.imread(basedir+x)), out)
    targ = map(lambda x: rgb2gray(plt.imread(basedir+x)), targ)

    return (np.array(inp), np.array(out), np.array(targ))

In [6]:
savedir = 'c:/Users/Airton/Desktop/run1eval'
basedir = 'c:/Users/Airton/Desktop/run1/'

In [7]:
def diffL1(x,y):
    """ L1 distance between 2 tensors """
    return np.sum(np.abs(x-y))*1.0/np.prod(x.shape)

In [8]:
# %matplotlib nbagg
def pyramidL2(img1, img2, show=False):
    """ 
    compare how similar two images are using pyrmid downscaling and RMSE 
    return: [(shape, rmse), .... ]
    """
    l2s = []
    for i,j in zip(transform.pyramid_gaussian(img1), transform.pyramid_gaussian(img2))[:6]:
        shape, rmse = i.shape, np.sqrt(measure.compare_mse(i,j))
        l2s.append((shape,rmse))
        if not show: continue
        ax, fig = plt.subplots(ncols=2,nrows=1)
        fig[0].imshow(i,cmap='gray')
        fig[0].set_xticks([])
        fig[0].set_yticks([])
        fig[1].imshow(i,cmap='gray')
        fig[1].set_xticks([])
        fig[1].set_yticks([])
        ax.text(.92, 0.4, 'dim: %s'%str(shape), fontsize=18)
        ax.text(.92, 0.6, 'RMSE: %s'%str(rmse)[:5], fontsize=18)
        ax.subplots_adjust(left=0.1)
#         ax.savefig(savedir+str(shape[0])+'.png', bbox_inches='tight')
        plt.show()
    return l2s
  
# print pyramidL2(out[0], targ[0], True)

In [9]:
def evaluateModel(rootpath):
    """ use RMSE an pyrmaid method to score a pix2pix model """
    _, mout, mtarg = readFolder(rootpath)
    allrmse = [pyramidL2(x,y) for x,y in zip(mout,mtarg)]
    avgByDim = {k[0]: 0.0 for k in allrmse[0]}
    for img in allrmse:
        for dim,rmse in img:
            avgByDim[dim] += rmse
    for k in avgByDim.iterkeys():     
        avgByDim[k] /= len(allrmse)
    return avgByDim

In [ ]:
# evaluate each of our models:
run1 = evaluateModel(basedir+'test_out/')

In [ ]:
sortkeys = sorted(old2.keys())
width = 0.1 
plt.xticks(np.arange(6), [str(k)[1:-1].replace(', ','x').replace('L','') for k in sortkeys])
for i, (model, desc) in enumerate(zip([old2, lowerThresh2, bands2and4, band6], ['Band 2 (non-normalized)', 'Band 2 (less-strict sparsity threshold)', 'Bands 2 and 4 (stacked)', 'Band 6'])):
#     plt.figure()
    plt.bar(np.arange(6)+width*i, [model[k] for k in sortkeys],width=width, label=desc)
plt.title("Models RMSE by dimension (eval on test data only)")
plt.legend()
plt.show()